In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-work


In [3]:
import sys
import numpy as np
import skimage
import cv2
import sklearn
import imp

In [4]:
import holoviews

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [5]:
import neukrill_net.utils
import neukrill_net.highlevelfeatures
import neukrill_net.stacked

In [6]:
import time

In [54]:
#%pdb

Automatic pdb calling has been turned OFF


In [7]:
settings = neukrill_net.utils.Settings('settings.json')

In [8]:
X,y = settings.flattened_train_paths(settings.classes)

In [9]:
pkl_names = ['pftas.pkl','contourhistogram.pkl','contourmoments.pkl','haralick.pkl']

In [10]:
t0 = time.time()
hlf = []
XF_list = []
for pkl_name in pkl_names:
    tmp = sklearn.externals.joblib.load('cache/'+pkl_name)
    hlf += [tmp[0]]
    XF_list += [tmp[1]]
print("Loading features took {}".format(time.time()-t0))

Loading features took 0.26828289032


In [11]:
XF = np.concatenate(XF_list,2)

In [12]:
XF.shape

(1, 30336, 366)

In [13]:
XF[0,0,:]

array([  9.27835052e-02,   8.24742268e-02,   1.28865979e-01,
         2.57731959e-01,   2.21649485e-01,   1.23711340e-01,
         5.67010309e-02,   1.03092784e-02,   2.57731959e-02,
         9.27835052e-02,   8.24742268e-02,   1.28865979e-01,
         2.57731959e-01,   2.21649485e-01,   1.23711340e-01,
         5.67010309e-02,   1.03092784e-02,   2.57731959e-02,
         7.88043478e-02,   8.69565217e-02,   1.35869565e-01,
         1.87500000e-01,   1.19565217e-01,   8.15217391e-02,
         1.22282609e-01,   1.03260870e-01,   8.42391304e-02,
         9.22432432e-01,   2.94594595e-02,   1.89189189e-02,
         1.70270270e-02,   1.00000000e-02,   1.62162162e-03,
         5.40540541e-04,   0.00000000e+00,   0.00000000e+00,
         9.22432432e-01,   2.94594595e-02,   1.89189189e-02,
         1.70270270e-02,   1.00000000e-02,   1.62162162e-03,
         5.40540541e-04,   0.00000000e+00,   0.00000000e+00,
         8.10550199e-01,   8.11117413e-02,   4.42427680e-02,
         3.14804311e-02,

## Naive Bayes

In [14]:
import sklearn.naive_bayes

In [15]:
clf = sklearn.naive_bayes.GaussianNB()

In [16]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=0.395761013031
Accuracy=0.185917721519
Logloss=26.1907140911


### Reduce with Feature selection

In [17]:
X_new = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=45).fit_transform(XF.squeeze(0), y)

In [18]:
my_X = X_new
clf = sklearn.naive_bayes.GaussianNB()

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=0.0630669593811
Accuracy=0.308742088608
Logloss=10.6894260686


## Random Forest

On original

In [19]:
import sklearn.ensemble

In [21]:
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=50.592176199
Accuracy=0.525250527426
Logloss=1.91537714433


This is similar to just the Contour Moments and Haralick features

On reduced

In [22]:
my_X = X_new

clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=23.738656044
Accuracy=0.502505274262
Logloss=1.94713154469


Does slightly worse with fewer features.

Maybe it was too few?

In [23]:
my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=35.6933050156
Accuracy=0.529733649789
Logloss=1.86473985616


### Hierarchical classifier

In [24]:
import neukrill_net.taxonomy

In [29]:
reload(neukrill_net.stacked)

<module 'neukrill_net.stacked' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/stacked.py'>

In [36]:
reload(neukrill_net.taxonomy)

<module 'neukrill_net.taxonomy' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/taxonomy.py'>

In [37]:
neukrill_net.taxonomy.taxonomy

{'no_class': {'artifacts': {}, 'artifacts_edge': {}},
 'plankton': {'chaetognaths': {'chaetognath_non_sagitta': {},
   'chaetognath_other': {},
   'chaetognath_sagitta': {}},
  'chordate_type1': {},
  'crustaceans': {'amphipods': {},
   'copepods': {'calanoid': {'copepod_calanoid': {},
     'copepod_calanoid_eggs': {},
     'copepod_calanoid_eucalanus': {},
     'copepod_calanoid_flatheads': {},
     'copepod_calanoid_frillyAntennae': {},
     'copepod_calanoid_large': {},
     'copepod_calanoid_large_side_antennatucked': {},
     'copepod_calanoid_octomoms': {},
     'copepod_calanoid_small_longantennae': {},
     'copepod_other': {}},
    'cyclopoid_copepods': {'copepod_cyclopoid_copilia': {},
     'oithona': {'copepod_cyclopoid_oithona': {},
      'copepod_cyclopoid_oithona_eggs': {}}}},
   'crustacean_other': {},
   'shrimp_like': {'decapods_all': {'decapods': {},
     'shrimp_caridean': {},
     'shrimp_sergestidae': {},
     'shrimp_zoea': {}},
    'euphausiids_all_ages': {'eupha

In [38]:
settings.classes

[u'acantharia_protist',
 u'acantharia_protist_big_center',
 u'acantharia_protist_halo',
 u'amphipods',
 u'appendicularian_fritillaridae',
 u'appendicularian_slight_curve',
 u'appendicularian_s_shape',
 u'appendicularian_straight',
 u'artifacts',
 u'artifacts_edge',
 u'chaetognath_non_sagitta',
 u'chaetognath_other',
 u'chaetognath_sagitta',
 u'chordate_type1',
 u'copepod_calanoid',
 u'copepod_calanoid_eggs',
 u'copepod_calanoid_eucalanus',
 u'copepod_calanoid_flatheads',
 u'copepod_calanoid_frillyAntennae',
 u'copepod_calanoid_large',
 u'copepod_calanoid_large_side_antennatucked',
 u'copepod_calanoid_octomoms',
 u'copepod_calanoid_small_longantennae',
 u'copepod_cyclopoid_copilia',
 u'copepod_cyclopoid_oithona',
 u'copepod_cyclopoid_oithona_eggs',
 u'copepod_other',
 u'crustacean_other',
 u'ctenophore_cestid',
 u'ctenophore_cydippid_no_tentacles',
 u'ctenophore_cydippid_tentacles',
 u'ctenophore_lobate',
 u'decapods',
 u'detritus_blob',
 u'detritus_filamentous',
 u'detritus_other',
 u'

In [39]:
marked_taxonomy = neukrill_net.stacked.propagate_labels_to_leaves(neukrill_net.taxonomy.taxonomy, settings.classes)

In [40]:
marked_taxonomy

{'no_class': {'artifacts': 8, 'artifacts_edge': 9},
 'plankton': {'chaetognaths': {'chaetognath_non_sagitta': 10,
   'chaetognath_other': 11,
   'chaetognath_sagitta': 12},
  'chordate_type1': 13,
  'crustaceans': {'amphipods': 3,
   'copepods': {'calanoid': {'copepod_calanoid': 14,
     'copepod_calanoid_eggs': 15,
     'copepod_calanoid_eucalanus': 16,
     'copepod_calanoid_flatheads': 17,
     'copepod_calanoid_frillyAntennae': 18,
     'copepod_calanoid_large': 19,
     'copepod_calanoid_large_side_antennatucked': 20,
     'copepod_calanoid_octomoms': 21,
     'copepod_calanoid_small_longantennae': 22,
     'copepod_other': 26},
    'cyclopoid_copepods': {'copepod_cyclopoid_copilia': 23,
     'oithona': {'copepod_cyclopoid_oithona': 24,
      'copepod_cyclopoid_oithona_eggs': 25}}},
   'crustacean_other': 27,
   'shrimp_like': {'decapods_all': {'decapods': 32,
     'shrimp_caridean': 92,
     'shrimp_sergestidae': 94,
     'shrimp_zoea': 95},
    'euphausiids_all_ages': {'euphausi

In [41]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=70.759239912
Logloss=1.8479429741
Time=17.3513498306
Accuracy=0.530129219409


### With all the features left in

In [42]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=87.1275148392
Logloss=1.91641586977
Time=20.142124176
Accuracy=0.526305379747


Try with a pipline to reduce the number of features at each level

In [44]:
import sklearn.pipeline

In [47]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=68.5872459412
Logloss=1.8722423589
Time=18.8261601925
Accuracy=0.520767405063


/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [47]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=68.5872459412
Logloss=1.8722423589
Time=18.8261601925
Accuracy=0.520767405063


/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## Logistic Regression

In [48]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

In [49]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=748.569411993
Accuracy=0.470266350211
Logloss=2.16218685973


In [50]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=145.000400066
Accuracy=0.527030590717
Logloss=1.83400284889


In [55]:
base_clf = sklearn.linear_model.LogisticRegression(random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=123.477735996
Logloss=2.53232405225
Time=0.659505844116
Accuracy=0.448641877637


In [52]:
base_clf = sklearn.linear_model.LogisticRegression(random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=21.4947209358
Logloss=2.04123557253
Time=1.12955403328
Accuracy=0.476331751055


## Linear SVC

In [56]:
clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=337.150140047
Accuracy=0.452202004219
Logloss=2.14135559841


In [57]:
clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=86.2040469646
Accuracy=0.553270042194
Logloss=1.7817047324


In [58]:
base_clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=1202.48394012
Logloss=2.14906265613
Time=122.273081064
Accuracy=0.434665084388


In [59]:
base_clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=177.089457035
Logloss=1.91130093883
Time=34.1300561428
Accuracy=0.488528481013


## Non-linear SVC

one-vs-one

In [60]:
clf = sklearn.svm.SVC(probability=True, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=518.949584007
Accuracy=0.494857594937
Logloss=1.82661110623


In [61]:
clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=121.488356829
Accuracy=0.522679324895
Logloss=1.70179310087


In [62]:
base_clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=446.839751005
Logloss=1.85225221785
Time=184.890357971
Accuracy=0.50039556962


In [63]:
base_clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=128.495127916
Logloss=1.79123650959
Time=55.6328210831
Accuracy=0.507318037975
